In [1]:
import pandas as pd

In [16]:
df = pd.read_csv('dataset.csv', index_col='order_id', parse_dates=True)

In [21]:
df.head(10)

,shop_id,user_id,order_amount,total_items,payment_method,created_at
order_id,,,,,,
1,53,746,224,2,cash,2017-03-13 12:36:56
2,92,925,90,1,cash,2017-03-03 17:38:52
3,44,861,144,1,cash,2017-03-14 4:23:56
4,18,935,156,1,credit_card,2017-03-26 12:43:37
5,18,883,156,1,credit_card,2017-03-01 4:35:11
6,58,882,138,1,credit_card,2017-03-14 15:25:01
7,87,915,149,1,cash,2017-03-01 21:37:57
8,22,761,292,2,cash,2017-03-08 2:05:38
9,64,914,266,2,debit,2017-03-17 20:56:50


In [18]:
df.describe(include='all')
# Here we can see that the $3145.13 is the mean of all order amounts. However the data is heavily skewed by a few orders
# of over 100 sneakers (probably by distributors). There are a few options that can be taken depending on the situation
# one of them is using the median, another is filtering out the outliers (ie. orders with over 100 sneakers) and 
# recalculating the mean

,shop_id,user_id,order_amount,total_items,payment_method,created_at
count,5000.000000,5000.000000,5000.000000,5000.00000,5000,5000
unique,NaN,NaN,NaN,NaN,3,4991
top,NaN,NaN,NaN,NaN,credit_card,2017-03-28 4:00:00
freq,NaN,NaN,NaN,NaN,1735,3
mean,50.078800,849.092400,3145.128000,8.78720,NaN,NaN
std,29.006118,87.798982,41282.539349,116.32032,NaN,NaN
min,1.000000,607.000000,90.000000,1.00000,NaN,NaN
25%,24.000000,775.000000,163.000000,1.00000,NaN,NaN
50%,50.000000,849.000000,284.000000,2.00000,NaN,NaN
75%,75.000000,925.000000,390.000000,3.00000,NaN,NaN


In [22]:
# Median AOV is $284 dollars, this is a much more reasonable amount:

df['order_amount'].median()

284.0

In [76]:
# Filtering out suspected distributor orders we can find the new average is $754. This is a much more reasonable number
# but still seems a little high. (We'll call the dataframe with the distributors filtered out df_no_dist)
df_no_dist = df[df['total_items']<=100]
df_no_dist.describe()

,shop_id,user_id,order_amount,total_items
count,4983.000000,4983.000000,4983.000000,4983.00000
mean,50.106362,849.918322,754.091913,1.99398
std,29.051718,86.800308,5314.092293,0.98318
min,1.000000,700.000000,90.000000,1.00000
25%,24.000000,776.000000,163.000000,1.00000
50%,50.000000,850.000000,284.000000,2.00000
75%,75.000000,925.000000,390.000000,3.00000
max,100.000000,999.000000,154350.000000,8.00000


In [71]:
# Looking at the maximum, we find there's an order for only 8 sneakers valued at $154,350. This is either a novelty
# item or a mistake. One option would be to filter out this order and recalculate the new mean.

df_no_dist.max()

shop_id                          100
user_id                          999
order_amount                  154350
total_items                        8
payment_method                 debit
created_at        2017-03-30 9:55:00
dtype: object

In [72]:
# But first we'll find the information for that transaction

df_no_dist.loc[df['order_amount'] == 154350]

,shop_id,user_id,order_amount,total_items,payment_method,created_at
order_id,,,,,,
692,78,878,154350,6,debit,2017-03-27 22:51:43


In [75]:
# Then we can see the statistics for that specific store to see if it their item is priced this high in fact or if it
# was a mistake. We find that it is not a mistake, the store must be selling a novelty sneakers.
# Their sneaker cost is $25,725.

df_no_dist.loc[df['shop_id'] == 78].describe()

,shop_id,user_id,order_amount,total_items
count,46.0,46.000000,46.000000,46.000000
mean,78.0,867.739130,49213.043478,1.913043
std,0.0,81.314871,26472.227449,1.029047
min,78.0,707.000000,25725.000000,1.000000
25%,78.0,812.500000,25725.000000,1.000000
50%,78.0,866.500000,51450.000000,2.000000
75%,78.0,935.750000,51450.000000,2.000000
max,78.0,997.000000,154350.000000,6.000000


In [81]:
# Depending on the purpose of our analysis, we may filter out this store to get a more realistic AOV
# across our stores. We will filter this out and call the new dataframe df_no_dist_or_nov

df_no_dist_or_nov = df_no_dist.loc[df['shop_id'] != 78]
df_no_dist_or_nov.describe()

,shop_id,user_id,order_amount,total_items
count,4937.000000,4937.000000,4937.000000,4937.000000
mean,49.846465,849.752279,302.580514,1.994734
std,29.061131,86.840313,160.804912,0.982821
min,1.000000,700.000000,90.000000,1.000000
25%,24.000000,775.000000,163.000000,1.000000
50%,50.000000,850.000000,284.000000,2.000000
75%,74.000000,925.000000,387.000000,3.000000
max,100.000000,999.000000,1760.000000,8.000000


In [82]:
# The AOV with these transactions filtered out is now $302, which, unsurprisingly is close to the median calculated
# above ($284)